<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="-unSkript-Runbooks-">unSkript Runbooks <a class="jp-InternalAnchorLink" href="#-unSkript-Runbooks-" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong><a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<strong>To stop idle EC2 instances using unSkript actions.</strong></div>
</center><center>
<h2 id="Delete-Unattached-EBS-Volume">Stop Idle EC2 Instances</h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1. AWS Find Idle Instances<br>2. Stop AWS Instances</p>

In [36]:
if instance_ids and not region:
    raise SystemExit("Enter AWS Region for given instances!")

<h3 id="AWS-List-All-Regions">AWS List All Regions</h3>
<p>In this action, we list all the available regions from AWS if the user does not provide a region as a parameter.</p>
<blockquote>
<p>Output variable: <code>region</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 --region us-west-2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="AWS-Find-Idle-Instances">AWS Find Idle Instances</h3>
<p>Here we will use unSkript&nbsp;<strong>AWS Find Idle Instances</strong> action. This action filters all idle instances from the given region, idle_cpu_threshold and idle_duration return a list of all the idle instances. It will execute if the <code>Instance_Ids</code>&nbsp;parameter is not passed.</p>
<blockquote>
<p>Input parameters: <code>region, idle_cpu_threshold, idle_duration</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>idle_instances</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from typing import Optional, Tuple
import pprint
import datetime
from beartype import beartype


@beartype
def aws_find_idle_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def is_instance_idle(instance_id , idle_cpu_threshold, idle_duration, cloudwatchclient):
    try:
        now = datetime.datetime.utcnow()
        start_time = now - datetime.timedelta(hours=idle_duration)
        cpu_utilization_stats = cloudwatchclient.get_metric_statistics(
            Namespace="AWS/EC2",
            MetricName="CPUUtilization",
            Dimensions=[{"Name": "InstanceId", "Value": instance_id}],
            StartTime=start_time.isoformat(),
            EndTime=now.isoformat(),
            Period=3600,
            Statistics=["Average"],
        )
        if not cpu_utilization_stats["Datapoints"]:
            return False
        average_cpu = sum([datapoint["Average"] for datapoint in cpu_utilization_stats["Datapoints"]]) / len(cpu_utilization_stats["Datapoints"])
    except Exception as e:
        raise e
    return average_cpu < idle_cpu_threshold

@beartype
def aws_find_idle_instances(handle, idle_cpu_threshold:int = 5, idle_duration:int = 6, region:str='') -> Tuple:
    """aws_find_idle_instances finds idle EC2 instances

    :type region: string
    :param region: AWS Region to get the instances from. Eg: "us-west-2"

    :type idle_cpu_threshold: int
    :param idle_cpu_threshold: (in percent) Idle CPU threshold (in percent)

    :type idle_duration: int
    :param idle_duration: (in hours) Idle CPU threshold (in hours)

    :rtype: Tuple with status result and list of Idle Instances.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2client = handle.client('ec2', region_name=reg)
            cloudwatchclient = handle.client("cloudwatch", region_name=reg)
            all_instances = ec2client.describe_instances()
            for instance in all_instances['Reservations']:
                for i in instance['Instances']:
                    if i['State']["Name"] == "running" and is_instance_idle(i['InstanceId'], reg, idle_cpu_threshold,idle_duration, cloudwatchclient ):
                        idle_instances = {}
                        idle_instances["instance"] = i['InstanceId']
                        idle_instances["region"] = reg
                        result.append(idle_instances)
        except Exception:
            pass
    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)





task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(outputName="idle_instances")

task.configure(inputParamsJson='''{
    "idle_cpu_threshold": "int(Idle_CPU_Threshold)",
    "idle_duration": "int(Idle_Duration)",
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not instance_ids",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_idle_instances, lego_printer=aws_find_idle_instances_printer, hdl=hdl, args=args)

<h3 id="Modify-Idle-Instances-Output">Modify Idle Instances Output</h3>
<p>In this action, we will pass the list of idle instances from Step 1 and sort the output as per Step 2.</p>
<blockquote>
<p>Output variable: <code>idle_instances_list</code></p>
</blockquote>

In [38]:
idle_instances_list = []
try:
    for key, value in idle_instances.items():
        if value[0] == False:
            if len(value[1])!= 0:
                idle_instances_list = value[1]
except Exception as e:
    if instance_ids:
        for instance in instance_ids:
            instance_dict = {}
            instance_dict["instance"] = instance
            instance_dict["region"] = region[0]
            idle_instances_list.append(instance_dict)
    else:
        raise Exception(e)
            

<h3 id="Stop-AWS-Instances">Stop AWS Instances</h3>
<p>Here we will use unSkript&nbsp;<strong>Stop AWS Instances</strong> action. In this action, we will pass the list of idle instances from step 1 and stop those instances.</p>
<blockquote>
<p>Input parameters: <code>instance_id</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>stop_instances</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_stop_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_stop_instances(handle, instance_id: str, region: str) -> Dict:
    """aws_stop_instances Stops instances.

        :type instance_id: string
        :param instance_id: String containing the name of AWS EC2 instance

        :type region: string
        :param region: AWS region for instance

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    output = {}
    res = ec2Client.stop_instances(InstanceIds=[instance_id])
    for instances in res['StoppingInstances']:
        output[instances['InstanceId']] = instances['CurrentState']

    return output


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_id": "iter.get(\\"instance\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "idle_instances_list",
    "iter_parameter": ["instance_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(idle_instances_list) != 0",
    "condition_result": true
    }''')
task.configure(outputName="stop_instances")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_stop_instances, lego_printer=aws_stop_instances_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to filter idle instances and stop those. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>